In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import networkx as nx

def generate_aether_data():
    np.random.seed(2026)
    
    # === 1. 参数设定 (Scale: Large) ===
    NUM_ISLANDS = 1000     # 节点规模
    CONNECTION_RADIUS = 12 # 只有距离小于此值的岛屿才有航线 (稀疏图)
    
    # === 2. 生成群岛节点 (Nodes) ===
    # 模拟 5 个主要群岛聚类 (Clusters)
    centers = [(20,20), (80,80), (20,80), (80,20), (50,50)] 
    coords = []
    
    for idx, center in enumerate(centers):
        # 高斯分布生成岛屿，模拟自然地理分布
        n_samples = NUM_ISLANDS // len(centers)
        locs = np.random.normal(loc=center, scale=10, size=(n_samples, 2))
        coords.append(locs)
    
    coords = np.vstack(coords)
    coords = np.clip(coords, 0, 100) # 限制在 0-100 坐标系内
    
    df_nodes = pd.DataFrame({
        'Node_ID': [f"Isl_{i:04d}" for i in range(NUM_ISLANDS)],
        'X': coords[:, 0],
        'Y': coords[:, 1],
        'Population': np.random.randint(100, 10000, NUM_ISLANDS),
        # 赋予部分节点高风险属性（模拟低海拔，可用于后续分析）
        'Elevation': np.random.uniform(0, 100, NUM_ISLANDS) 
    })
    
    # === 3. 生成航线边表 (Edges) - 动态权重 ===
    print("🌊 正在构建航线网络 (可能需要几秒钟)...")
    dist_matrix = distance.cdist(coords, coords, 'euclidean')
    
    sources = []
    targets = []
    base_costs = [] 
    storm_costs = [] 
    
    # 构建稀疏图：只连接地理上相近的岛屿
    rows, cols = np.where(np.triu(dist_matrix < CONNECTION_RADIUS, k=1))
    
    for i, j in zip(rows, cols):
        d = dist_matrix[i, j]
        sources.append(df_nodes.iloc[i]['Node_ID'])
        targets.append(df_nodes.iloc[j]['Node_ID'])
        base_costs.append(d)
        
        # [动态性核心]: 
        # 风暴对长距离航线(Open Sea)的影响呈指数级增长
        # 对短距离航线(Coastal)影响较小
        if d > CONNECTION_RADIUS * 0.6:
            surge_factor = np.random.uniform(2.5, 5.0) # 深海巨浪
        else:
            surge_factor = np.random.uniform(1.1, 1.5) # 沿岸风浪
            
        storm_costs.append(d * surge_factor)

    df_edges = pd.DataFrame({
        'Source': sources,
        'Target': targets,
        'Base_Cost': base_costs,   # 平静海况权重
        'Storm_Cost': storm_costs  # 风暴海况权重
    })
    
    # 确保图连通 (连接孤岛到最近邻居，防止 TSP 无解)
    G = nx.from_pandas_edgelist(df_edges, 'Source', 'Target', 'Base_Cost')
    if not nx.is_connected(G):
        print("🔧 修复网络孤岛...")
        components = list(nx.connected_components(G))
        main_comp = max(components, key=len)
        for comp in components:
            if comp == main_comp: continue
            # 将孤岛连接到主岛
            u = list(comp)[0]
            v = list(main_comp)[np.random.randint(len(main_comp))]
            df_edges.loc[len(df_edges)] = [u, v, 50.0, 100.0] # 添加昂贵的应急航线

    # 保存
    df_nodes.to_csv('Island_Nodes.csv', index=False)
    df_edges.to_csv('Sea_Routes.csv', index=False)
    
    print(f"✅ [Project Aether] 数据集生成完毕")
    print(f"   Nodes: {len(df_nodes)}")
    print(f"   Edges: {len(df_edges)}")
    print(f"   Dynamic Feature: 'Base_Cost' vs 'Storm_Cost'")

if __name__ == "__main__":
    generate_aether_data()